In [5]:
from selenium import webdriver as wd
import numpy as np
import pandas as pd
import json
import openpyxl as pyxl
import os
import requests
import datetime

url = 'https://www.law360.com/'

with open("./Login.json","r") as f:
    json_user = json.load(f)

user_id = json_user['UserName']
user_pw = json_user['pw']

driver = wd.Chrome('./chromedriver')
driver.implicitly_wait(1)
driver.get(url)
driver.implicitly_wait(1)

signin = driver.find_element_by_xpath('//*[@id="navigation_new"]/div[2]/div[1]/span/a')
signin.click()
driver.implicitly_wait(1)

email = driver.find_element_by_xpath('//*[@id="email"]')
pw = driver.find_element_by_xpath('//*[@id="password"]')
submit = driver.find_element_by_xpath('//*[@id="btnSignIn"]')

email.send_keys(user_id)
pw.send_keys(user_pw)
submit.click()
driver.implicitly_wait(1)

In [6]:
def mkLower (A):
    a = A.lower()
    return a

def writeExcel(exactCase):
    path = "./result.xlsx"
    if(os.path.isfile(path) == False):
        wb = pyxl.Workbook()
        ws = wb.active
        ws.append(["Case Name", "Case Start Date", "Nature of Suit | Court"])

        for ec in exactCase:
            ws.append([ec.find_element_by_tag_name("a").get_attribute('title'), ec.find_element_by_class_name('dtstamp').get_attribute('title'), ec.find_element_by_class_name('entry-content').text])
    else:
        wb = pyxl.load_workbook(path, data_only=True)
        ws = wb.active
        
        for ec in exactCase:
            ws.append([ec.find_element_by_tag_name("a").get_attribute('title'), ec.find_element_by_class_name('dtstamp').get_attribute('title'), ec.find_element_by_class_name('entry-content').text])
        
        
    wb.save("./result.xlsx")
        
    print('writeExcel function is finished')

def Jump2Activity (Case):
    print('working good')

def Jump2Case (caseList):
    print('working goood')
    

In [7]:
def srchFunc (srchTable):


    for i in range(len(srchTable)):
        
        srchTerm = srchTable['Client'][i] + " v. "+ srchTable['Opposite'][i]
        client = mkLower(srchTable['Client'][i])
        opposite = mkLower(srchTable['Opposite'][i])
        Type_Loc = srchTable['Type'][i] + ' | ' + srchTable['Location'][i]
        Type_Loc = mkLower(Type_Loc)
        
        print("고객 = {}, 상대측 = {}, 타입과 위치 = {}".format(client, opposite, Type_Loc))
        
        print(srchTerm)
        srchInput = driver.find_element_by_xpath('//*[@id="global_search_field"]')
        srchBtn = driver.find_element_by_xpath('//*[@id="btnSearchResults"]')
        srchInput.clear()
        srchInput.send_keys(srchTerm)
        srchBtn.click()
        driver.implicitly_wait(1)
        CasesTab = driver.find_element_by_xpath('//*[@id="results"]/ul/li[2]/a')
        CasesTab.click()
        driver.implicitly_wait(100)
        
        tab_content = driver.find_element_by_class_name('tab-content')
        result_Num = tab_content.find_element_by_tag_name('h2').text.strip("Case (").strip(")")
                
        if(int(result_Num) > 20):
            pageSize_200 = driver.find_element_by_css_selector('#selectPageSize > option:nth-child(4)')
            pageSize_200.click()
            
        tab_content = driver.find_element_by_class_name('tab-content')
        list_Table = tab_content.find_elements_by_class_name('result')
        
        if(int(result_Num) == len(list_Table)):
            print("화면 리스트 {} = 총 결과 {}" .format(int(result_Num), len(list_Table)))
        
        exact_Case = list()
        
        for li in list_Table:
            Case_Name = li.find_element_by_tag_name("a").get_attribute('title')
            Case_Name = mkLower(Case_Name)
            
            if((Case_Name.count(client) * Case_Name.count(opposite)) != 0):
                if(mkLower(li.find_element_by_class_name('entry-content').text).count(Type_Loc) !=0):
                    exact_Case.append(li)
                    
                    
        print("일치하는 이름의 Case 수: {}" .format(len(exact_Case)))
        for ec in exact_Case:
            print(ec.find_element_by_tag_name("a").get_attribute('title'))
            print(ec.find_element_by_css_selector('p > strong').text)

        driver.implicitly_wait(1)
        writeExcel(exact_Case)
        
        


In [8]:
srchTable = pd.read_excel('./searchTable.xlsx')
srchTable = srchTable.replace(np.nan, '', regex=True)

print(srchTable)
len(srchTable)

srchFunc(srchTable)




                Client  Opposite    Type  Location
0        SK Innovation   LG Chem          Delaware
1  Samsung Electronics   Elm 3DS  Patent          
2  Samsung Electronics  Ericsson  Patent          
고객 = sk innovation, 상대측 = lg chem, 타입과 위치 =  | delaware
SK Innovation v. LG Chem
화면 리스트 4 = 총 결과 4
일치하는 이름의 Case 수: 3
SK Innovation Co., Ltd. v. LG Chem, LTD, et al
Patent
SK Innovation Co., Ltd. v. LG Chem, Ltd., et al
Patent
LG Chem, Ltd. et al v. SK Innovation Co., Ltd. et al
Other Statutory Actions
writeExcel function is finished
고객 = samsung electronics, 상대측 = elm 3ds, 타입과 위치 = patent | 
Samsung Electronics v. Elm 3DS
화면 리스트 2 = 총 결과 2
일치하는 이름의 Case 수: 1
Elm 3DS Innovations LLC v. Samsung Electronics Co. Ltd. et al
Patent
writeExcel function is finished
고객 = samsung electronics, 상대측 = ericsson, 타입과 위치 = patent | 
Samsung Electronics v. Ericsson
화면 리스트 67 = 총 결과 67
일치하는 이름의 Case 수: 5
Ericsson, Inc. et al v. Samsung Electronics Co., Ltd. et al
Patent
Ericsson Inc. et al v. SAMSUNG E